In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='52KYU1B4HW8FCKX0')

In [2]:
class ScriptData:

    def __init__(self, fetched_data = {}, converted_data = {}):
        self.fetched_data = fetched_data
        self.converted_data = converted_data

    def __getitem__(self, script, datatype='converted'):
        if datatype == 'converted':
            try:
                return self.converted_data[script]
            except KeyError:
                return 'converted intraday data not available'

        elif datatype == 'fetched':
            try:
                return self.fetched_data[script]
            except KeyError:
                return

    def __setitem__(self, script, data, datatype):
        if datatype == 'fetched':
            self.fetched_data[script] = data
        elif datatype == 'converted':
            self.converted_data[script] = data

    def __contains__(self, script):
        if script in self.converted_data:
            return True
        return False

    def fetch_intraday_data(self, script):
        data, metadata = ts.get_intraday(symbol=script)
        self.__setitem__(script, data, 'fetched')

    def convert_intraday_data(self, script):
        data = self.__getitem__(script, 'fetched')
        if data is None:
            return 'fetch data for this stock first'

        # Arranging dataframe
        df = pd.DataFrame(data).T
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df.insert(0, 'timestamp', df.index)
        df.index = np.arange(len(data))

        # Adopting appropriate datatypes
        df = df.astype({'open': 'float', 'high': 'float', 'low': 'float', 'close': 'float', 'volume': 'int'})
        df['timestamp'] = pd.to_datetime(df.timestamp)

        self.__setitem__(script, df, 'converted')

In [3]:
script_data = ScriptData()

In [4]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,97.0000,97.1200,97.000,97.1000,23421
1,2023-02-15 19:45:00,96.9399,97.0000,96.900,96.9800,24896
2,2023-02-15 19:30:00,97.0000,97.0100,96.910,96.9400,12484
3,2023-02-15 19:15:00,96.9300,97.0100,96.800,97.0000,23284
4,2023-02-15 19:00:00,96.9800,97.0000,96.930,96.9400,17077
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,92.6600,93.0399,92.530,93.0350,1306004
96,2023-02-14 12:00:00,92.3300,92.7300,92.270,92.6607,1437391
97,2023-02-14 11:45:00,92.7400,92.7400,92.260,92.3531,1869190
98,2023-02-14 11:30:00,93.1400,93.2500,92.695,92.7300,1506119


In [5]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.4399,155.510,155.4399,155.4800,17287
1,2023-02-15 19:45:00,155.4600,155.500,155.4000,155.4000,19333
2,2023-02-15 19:30:00,155.4800,155.490,155.4400,155.4700,9288
3,2023-02-15 19:15:00,155.3100,155.500,155.3000,155.4800,18221
4,2023-02-15 19:00:00,155.3900,155.390,155.3200,155.3300,7452
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,151.6950,152.120,151.5700,151.9700,1337676
96,2023-02-14 12:00:00,151.1300,151.830,151.1000,151.7000,1509483
97,2023-02-14 11:45:00,151.5500,152.030,151.0400,151.1400,2000947
98,2023-02-14 11:30:00,152.0300,152.590,151.4400,151.5418,1951865


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    new_df = df.loc[:,['timestamp']]
    close_series = df.loc[:, 'close']

    # Using rolling function to calculate mean of previous 'timeperiod' number of rows in close column
    rolling_average = close_series.rolling(window=timeperiod, min_periods=timeperiod).mean()

    new_df.insert(1, 'indicator', rolling_average)
    return new_df

In [10]:
indicator1(script_data['GOOGL'], 5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:45:00,NaN
2,2023-02-15 19:30:00,NaN
3,2023-02-15 19:15:00,NaN
4,2023-02-15 19:00:00,96.99200
...,...,...
95,2023-02-14 12:15:00,93.27226
96,2023-02-14 12:00:00,93.05054
97,2023-02-14 11:45:00,92.87866
98,2023-02-14 11:30:00,92.76234


In [11]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:45:00,NaN
2,2023-02-15 19:30:00,NaN
3,2023-02-15 19:15:00,NaN
4,2023-02-15 19:00:00,155.43200
...,...,...
95,2023-02-14 12:15:00,152.13704
96,2023-02-14 12:00:00,151.92404
97,2023-02-14 11:45:00,151.73104
98,2023-02-14 11:30:00,151.64834


In [12]:
class Strategy:

    def __init__(self, script, intraday_data={}):
        self.script = script
        self.intraday_data = intraday_data
    
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.intraday_data = script_data[self.script]

    def get_signals(self):
        indicator_data = indicator1(self.intraday_data, 5)
        indicator_data['close_data'] = self.intraday_data.loc[:, 'close']
    
        # BUY when following indicator_data is greater than following close_data and previous indicator_data is less than previous close_data
        buy_condition = (indicator_data['indicator'].shift() > indicator_data['close_data'].shift()) & (indicator_data['indicator'].shift(-1) < indicator_data['close_data'].shift(-1))

        # SELL when previous indicator_data is greater than previous close_data and following indicator_data is less than following close_data
        sell_condition = (indicator_data['indicator'].shift(-1) > indicator_data['close_data'].shift(-1)) & (indicator_data['indicator'].shift() < indicator_data['close_data'].shift())

        # NO SIGNAL otherwise
        indicator_data['signal'] = np.select([buy_condition, sell_condition, True], ['BUY', 'SELL', 'NO SIGNAL'])

        # New dataframe 'signals' with columns 'timestamp' and 'signal' tracking only BUY/SELL rows 
        signals = indicator_data.loc[:, ['timestamp', 'signal']]
        signals.drop(signals[signals.signal=='NO SIGNAL'].index, inplace = True)
        signals.reset_index(drop=True, inplace = True)
    
        return signals

In [13]:
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2023-02-15 18:00:00,BUY
1,2023-02-15 17:45:00,BUY
2,2023-02-15 17:30:00,SELL
3,2023-02-15 17:15:00,SELL
4,2023-02-15 17:00:00,BUY
5,2023-02-15 16:45:00,BUY
6,2023-02-15 16:00:00,SELL
7,2023-02-15 15:45:00,SELL
8,2023-02-15 15:00:00,BUY
9,2023-02-15 14:45:00,BUY
